In [ ]:
# default_exp hmmer

In [ ]:
#export
import pyhmmer
import shlex, subprocess
import os

In [ ]:
#hide
def build_hmm(alnFile):
    abc = pyhmmer.easel.Alphabet.amino()
    builder = pyhmmer.plan7.Builder(alphabet=abc)

    with pyhmmer.easel.MSAFile(alnFile) as msa_file:
        msa_file.set_digital(abc)
        msa = next(msa_file)

    # MSA must have a name, otherwise building will fail
    if msa.name is None:
        msa.name = b"alignment"
    builder = pyhmmer.plan7.Builder(abc)
    background = pyhmmer.plan7.Background(abc)
    hmm, _, _ = builder.build_msa(msa, background)

    # hmm, _, _ = builder.build_msa(msa, Background(abc))

    return hmm

In [ ]:
#export
def alnFile2hmmFile(alnFile):
	devnull = open(os.devnull, 'w')
	hmmFile = alnFile+".hmm"
	call_list = ''.join(['hmmbuild ',hmmFile,' ', alnFile])   
	commands = shlex.split(call_list)  
	subprocess.Popen(commands, stdin=subprocess.PIPE, stderr=subprocess.PIPE,stdout=devnull).communicate()
	
	return hmmFile

In [ ]:
#hide
build_hmm("data/enolase.part001.fa.aln")

In [ ]:
#hide
test_alnFile2hmmFile = alnFile2hmmFile("data/enolase.part001.fa.aln")
test_alnFile2hmmFile

'data/enolase.part001.fa.aln.hmm'

In [ ]:
#export
def load_hmm(hmmFile):
    with pyhmmer.plan7.HMMFile(hmmFile) as h:
        hmm = next(h)
    return hmm

In [ ]:
#hide
test_hmm = load_hmm(test_alnFile2hmmFile)

In [ ]:
#export
def run_hmmsearch(queryFile, alnFile=None, hmmFile=None, writeHMM=True):
    if writeHMM:
        if alnFile != None:
            hmmFile = alnFile2hmmFile(alnFile)
        hmm = load_hmm(hmmFile)
    else:
        hmm = build_hmm(alnFile)

    with pyhmmer.easel.SequenceFile(queryFile) as seq_file:
        sequences = [ seq.digitize(hmm.alphabet) for seq in seq_file ]
    pipeline = pyhmmer.plan7.Pipeline(hmm.alphabet)
    hits = pipeline.search_hmm(hmm, sequences) # Has lots of goodies!
    return hits


In [ ]:
#hide
test_run_hmmer = run_hmmer("data/enolase.part002.fa",test_hmm)

In [ ]:
#hide
test_run_hmmer